# 6.1 パラメータチューニング

## 6.1.1 ハイパーパラメータの探索手法

1. 手動
2. グリッドサーチ/ランダムサーチ  
   パラメータ空間を予め指定した範囲を規則的に探索/予め指定した手法でランダムに探索  
   `sckikit-learn.model_selection` の `GridSearchCV`, `RandomizedSearchCV` など
3. ベイズ最適化 (Bayseian Optimization)  
    以前に計算したパラメータの履歴に基づいてベイズの手法を用いて選択  
    `hyperopt`, `optuna` など

kaggle でのパラメータ探索は手で行っている人が多い印象。

## 6.1.2 パラメータチューニングで設定すること

1. ベースラインとなるパラメータ
2. 探索する対象となるパラメータとその範囲
3. 手動で調節するか、自動的に探索するか
4. 評価の枠組み (fold の分け方など)

パラメータを自動で調節する場合には、

- パラメータチューニングをしすぎて学習データに過剰に適合してしまう
- 計算時間が長くなる

といった問題が起こり得るので、

- チューニングとモデルの作成をする際の fold の分け方を変える
- validation の fold の 1 つだけを用いて精度を確認する

などの対策をしたほうが良い。

## 6.1.3 パラメータチューニングのポイント

各モデルには結果を大きく左右する大事なパラメータが存在する。  
そのため、パラメータを探索する際はその**モデルの重要なパラメータから調節していく**ことが大事である。  

また、**モデルとパラメータの関係を理解して得られた結果からなぜそうなったのかを理解すること**で、次にどうパラメータを変化させるべきかを考えながらチューニングしていくとよい。

なお、GBDT ではパラメータチューニングよりも良い特徴量を追加するほうが有用なことが多いので、あまりチューニングに時間を割かない方が良い。


## 6.1.4 ベイズ最適化でのパラメータ探索

Tree-structured Parzen Estimator (TPE) というアルゴリズムを用いて最適化を行っている `hyperopt` と `optuna` について、以下具体的な使い方を見ていく。

In [33]:
# ---------------------------------
# データ等の準備
# ----------------------------------
import numpy as np
import pandas as pd

# train_xは学習データ、train_yは目的変数、test_xはテストデータ
# pandasのDataFrame, Seriesで保持します。（numpyのarrayで保持することもあります）

train = pd.read_csv('../input/sample-data/train_preprocessed.csv')
train_x = train.drop(['target'], axis=1)
train_y = train['target']
test_x = pd.read_csv('../input/sample-data/test_preprocessed.csv')

# 学習データを学習データとバリデーションデータに分ける
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(train_x))[0]  # 最初の fold のみ用いる
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

# xgboostによる学習・予測を行うクラス
import xgboost as xgb


class Model:

    def __init__(self, params=None):
        self.model = None
        if params is None:
            self.params = {}
        else:
            self.params = params

    def fit(self, tr_x, tr_y, va_x, va_y):
        params = {'objective': 'binary:logistic', 'eval_metric': 'error', 'verbosity': 1, 'random_state': 71}
        params.update(self.params)
        num_round = 10
        dtrain = xgb.DMatrix(tr_x, label=tr_y)
        dvalid = xgb.DMatrix(va_x, label=va_y)
        watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
        self.model = xgb.train(params, dtrain, num_round, verbose_eval=False, evals=watchlist)

    def predict(self, x):
        data = xgb.DMatrix(x)
        pred = self.model.predict(data)
        return pred

### [`hyperopt`](http://hyperopt.github.io/hyperopt/)

具体的な手順は以下のよう。

1. 最小化したい評価指標を返す関数を作成する (`score` function) 
2. 探索するパラメータ範囲を定義する (`space` 変数)
3. 探索回数を指定する (`max_eval` 変数)

以上を `hyperopt.fmin` 関数に代入して探索を行う。

経験的には 25 回程度の探索でそれなりに妥当なパラメータが見つかり始め、100 回程度で十分な探索が行われるようである。


In [34]:
# -----------------------------------
# hyperoptを使ったパラメータ探索
# -----------------------------------
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import log_loss


def score(params):
    # パラメータを与えたときに最小化する評価指標を指定する
    # 具体的には、モデルにパラメータを指定して学習・予測させた場合のスコアを返すようにする

    # max_depthの型を整数型に修正する
    params['max_depth'] = int(params['max_depth'])

    # Modelクラスを定義しているものとする
    # Modelクラスは、fitで学習し、predictで予測値の確率を出力する
    model = Model(params)
    model.fit(tr_x, tr_y, va_x, va_y)
    va_pred = model.predict(va_x)
    score = log_loss(va_y, va_pred)
    print(f'params: {params}, logloss: {score:.4f}')

    # 情報を記録しておく
    history.append((params, score))

    return {'loss': score, 'status': STATUS_OK}


# 探索するパラメータの空間を指定する
# hp.choiceでは、複数の選択肢から選ぶ
# hp.uniformでは、下限・上限を指定した一様分布から抽出する。引数は下限・上限
# hp.quniformでは、下限・上限を指定した一様分布のうち一定の間隔ごとの点から抽出する。引数は下限・上限・間隔
# hp.loguniformでは、下限・上限を指定した対数が一様分布に従う分布から抽出する。引数は下限・上限の対数をとった値
space = {
    'min_child_weight': hp.quniform('min_child_weight', 1, 5, 1),
    'max_depth': hp.quniform('max_depth', 3, 9, 1),
    'gamma': hp.quniform('gamma', 0, 0.4, 0.1),
}

# hyperoptによるパラメータ探索の実行
max_evals = 20
trials = Trials()
history = []
fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=max_evals)

# 記録した情報からパラメータとスコアを出力する
# （trialsからも情報が取得できるが、パラメータの取得がやや行いづらいため）
history = sorted(history, key=lambda tpl: tpl[1])
best = history[0]
print(f'best params:{best[0]}, score:{best[1]:.4f}')

params: {'gamma': 0.30000000000000004, 'max_depth': 4, 'min_child_weight': 2.0}, logloss: 0.3221
params: {'gamma': 0.1, 'max_depth': 8, 'min_child_weight': 2.0}, logloss: 0.2851
params: {'gamma': 0.1, 'max_depth': 4, 'min_child_weight': 4.0}, logloss: 0.3267
params: {'gamma': 0.1, 'max_depth': 7, 'min_child_weight': 2.0}, logloss: 0.2880
params: {'gamma': 0.30000000000000004, 'max_depth': 8, 'min_child_weight': 2.0}, logloss: 0.2849
params: {'gamma': 0.2, 'max_depth': 5, 'min_child_weight': 4.0}, logloss: 0.3147
params: {'gamma': 0.2, 'max_depth': 9, 'min_child_weight': 2.0}, logloss: 0.2792
params: {'gamma': 0.30000000000000004, 'max_depth': 4, 'min_child_weight': 4.0}, logloss: 0.3267
params: {'gamma': 0.1, 'max_depth': 7, 'min_child_weight': 4.0}, logloss: 0.2942
params: {'gamma': 0.30000000000000004, 'max_depth': 3, 'min_child_weight': 4.0}, logloss: 0.3543
params: {'gamma': 0.30000000000000004, 'max_depth': 7, 'min_child_weight': 4.0}, logloss: 0.2940
params: {'gamma': 0.2, 'max_d

### `optuna`

2018 年末に公開されたフレームワークで、最適化のアルゴリズムに TPE を用いているのは hyperopt と同じだが、
API が使いやすく、そしてより効率的になっている[<sup>1</sup>](#fn1)。

個人的には最近よく聞くのはこっちな気がする。

<span id="fn1"> https://optuna.readthedocs.io/en/stable/ </span>

In [37]:
import optuna

def objective(trial):

    params = {
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'gamma': trial.suggest_uniform('gamma', 0, 0.4),
    }
    model = Model(params)
    model.fit(tr_x, tr_y, va_x, va_y)
    va_pred = model.predict(va_x)
    score = log_loss(va_y, va_pred)

    print(f'params: {params}, logloss: {score:.4f}')

    return score

study = optuna.create_study()
study.optimize(objective, n_trials=20)

print(f'best params:{study.best_params}, score:{study.best_value:.4f}')

[I 2021-10-03 09:40:06,451] A new study created in memory with name: no-name-f643c0b5-0daf-479b-b6a0-ebf0bc35e1f7
[I 2021-10-03 09:40:06,543] Trial 0 finished with value: 0.3147349045306444 and parameters: {'min_child_weight': 4, 'max_depth': 5, 'gamma': 0.13116261102965884}. Best is trial 0 with value: 0.3147349045306444.
[I 2021-10-03 09:40:06,594] Trial 1 finished with value: 0.3542746682226658 and parameters: {'min_child_weight': 1, 'max_depth': 3, 'gamma': 0.3284449296487035}. Best is trial 0 with value: 0.3147349045306444.
[I 2021-10-03 09:40:06,692] Trial 2 finished with value: 0.30089185103923083 and parameters: {'min_child_weight': 1, 'max_depth': 6, 'gamma': 0.04387353487753463}. Best is trial 2 with value: 0.30089185103923083.


params: {'min_child_weight': 4, 'max_depth': 5, 'gamma': 0.13116261102965884}, logloss: 0.3147
params: {'min_child_weight': 1, 'max_depth': 3, 'gamma': 0.3284449296487035}, logloss: 0.3543
params: {'min_child_weight': 1, 'max_depth': 6, 'gamma': 0.04387353487753463}, logloss: 0.3009


[I 2021-10-03 09:40:06,851] Trial 3 finished with value: 0.28736135601624846 and parameters: {'min_child_weight': 2, 'max_depth': 8, 'gamma': 0.04928800677802361}. Best is trial 3 with value: 0.28736135601624846.
[I 2021-10-03 09:40:06,961] Trial 4 finished with value: 0.291297506006062 and parameters: {'min_child_weight': 3, 'max_depth': 7, 'gamma': 0.04807251312640477}. Best is trial 3 with value: 0.28736135601624846.


params: {'min_child_weight': 2, 'max_depth': 8, 'gamma': 0.04928800677802361}, logloss: 0.2874
params: {'min_child_weight': 3, 'max_depth': 7, 'gamma': 0.04807251312640477}, logloss: 0.2913


[I 2021-10-03 09:40:07,099] Trial 5 finished with value: 0.28764107541665435 and parameters: {'min_child_weight': 2, 'max_depth': 8, 'gamma': 0.22122485704736428}. Best is trial 3 with value: 0.28736135601624846.
[I 2021-10-03 09:40:07,161] Trial 6 finished with value: 0.3228588599473238 and parameters: {'min_child_weight': 5, 'max_depth': 4, 'gamma': 0.18875010745446677}. Best is trial 3 with value: 0.28736135601624846.
[I 2021-10-03 09:40:07,210] Trial 7 finished with value: 0.3542746682226658 and parameters: {'min_child_weight': 2, 'max_depth': 3, 'gamma': 0.20096694125766287}. Best is trial 3 with value: 0.28736135601624846.


...

params: {'min_child_weight': 2, 'max_depth': 9, 'gamma': 0.3995490529163783}, logloss: 0.2809
params: {'min_child_weight': 4, 'max_depth': 7, 'gamma': 0.2800725105935713}, logloss: 0.2940


[I 2021-10-03 09:40:08,816] Trial 17 finished with value: 0.2805261413246393 and parameters: {'min_child_weight': 3, 'max_depth': 9, 'gamma': 0.34452935506123883}. Best is trial 17 with value: 0.2805261413246393.
[I 2021-10-03 09:40:08,918] Trial 18 finished with value: 0.3012499739944935 and parameters: {'min_child_weight': 3, 'max_depth': 6, 'gamma': 0.3451364003050326}. Best is trial 17 with value: 0.2805261413246393.


params: {'min_child_weight': 3, 'max_depth': 9, 'gamma': 0.34452935506123883}, logloss: 0.2805
params: {'min_child_weight': 3, 'max_depth': 6, 'gamma': 0.3451364003050326}, logloss: 0.3012


[I 2021-10-03 09:40:09,069] Trial 19 finished with value: 0.28517025641947985 and parameters: {'min_child_weight': 2, 'max_depth': 8, 'gamma': 0.12198721505975081}. Best is trial 17 with value: 0.2805261413246393.


params: {'min_child_weight': 2, 'max_depth': 8, 'gamma': 0.12198721505975081}, logloss: 0.2852
best params:{'min_child_weight': 3, 'max_depth': 9, 'gamma': 0.34452935506123883}, score:0.2805


ただ、ベイズ最適化によるチューニングでは以下のような問題が生じることがある。

- 計算時間のかかりすぎる試行  
  学習率を小さくした場合は学習がなかなか進まなくなってしまうので、事前に調整しておくなどの対策が必要。
- パラメータ間の依存性
  チューニングされるパラメータ同士は完全に独立でないため、依存性が強く表れる場合は効率的な探索ができない可能性がある。  
  パラメータ空間を明示的にするか、試行回数を増やす必要がある。
- 評価のランダム性によるばらつき
  評価のぶれが大きいときは効果的に探索できないので、cross validation の平均で評価するなど試行回数を増やすなどの必要がある。

従って、ベイズ最適化を使う場合はこれらの点に注意すること。

## 5.1.5 GBDT のパラメータおよびそのチューニング

xgboost のパラメータ

<center>
|      parameter     | explanation |
|:------------------:|:-----------:|
|        `eta`       | 学習率      |
|     `num_round`    |             |
|     `max_depth`    |             |
| `min_child_weight` |             |
|       `gamma`      |             |
| `colsample_bytree` |             |
|     `subsample`    |             |
|       `alpha`      |             |
|      `lambda`      |             |

<\center>

具体的なチューニングの例は p.321 - の COLUMN を参考に。

## 5.1.6 ニューラルネットのパラメータおよびそのチューニング



具体的なチューニングの例は p.321 - の COLUMN を参考に。


## 5.1.7 線形モデルのパラメータおよびそのチューニング

線形モデルでは正則化のパラメータがチューニング対象となる。  
対象のパラメータが少なく計算も比較的早いので、広範囲を探索することが可能である。

Ref. [正則化（Ridge,Lasso）](https://qiita.com/greatonbi/items/0322d420af46d3ed9183)